In [1]:
from datetime import datetime
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm
from time import sleep
import os
from bs4 import BeautifulSoup
import random
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

#import sys 
#sys.path.append("../")   aquí no es necesario porque tengo el src a la misma altura que el jupyter

# from src.funciones import info_vuelos, crearDF_formateado, info_hoteles, crearDF_formateado_hoteles

In [34]:
driver = webdriver.Chrome()
url_wunder = "https://www.guias.travel/tour/"
driver.get(url_wunder)
driver.maximize_window()
driver.implicitly_wait(6)

try: 
    driver.find_element("css selector", "#search-form-input").click()
except:
    print("No encuentro el botón")

driver.find_element("css selector", "#search-form-input").send_keys("santa cruz de tenerife", Keys.ENTER)
for i in range(5):  
    driver.execute_script("window.scrollBy(0, 1000);")  
    sleep(1)  
try: 
    driver.find_element("css selector", "#more_tours > span").click()
except:
    print("No encuentro el botón")

for i in range(5):  
    driver.execute_script("window.scrollBy(0, 1000);") 
    sleep(1)  
driver.find_element("css selector", "#moove_gdpr_cookie_info_bar > div > div > div.moove-gdpr-button-holder > button.mgbutton.moove-gdpr-infobar-reject-btn.gdpr-fbo-1").click()

try: 
    sleep(2)
    driver.find_element("css selector", "#more_tours > span").click()
    driver.find_element("css selector", "#more_tours > span").click()
except:
    print("No encuentro el botón")
for i in range(1):  
    driver.execute_script("window.scrollBy(0, 1000);") 
    sleep(1)  

No encuentro el botón


*Hago scroll para que me salgan todas las opciones de la página*

In [35]:
sopa_actividades_tene = BeautifulSoup(driver.page_source)
lista_actividades_tene = sopa_actividades_tene.find_all("div", {"class": "strip_all_tour_list"})

actividades_tene = [actividad.find("h2").getText() for actividad in lista_actividades_tene]
categoria_activides_tene= [actividad.find("h3").getText() for actividad in lista_actividades_tene]
descripcion_actividad_tene = [actividad.find("p").getText() for actividad in lista_actividades_tene]


Comprobación de que tienen el mismo tamaño

In [7]:
print(len(actividades_tene))
print(len(categoria_activides_tene))
print(len(descripcion_actividad_tene))

20
20
20


In [36]:
lista_precios_tene = sopa_actividades_tene.find_all("div", {"class": "price_container"})
precio_actividades_tene = [precio.getText() for precio in lista_precios_tene]


In [9]:
# lista_precios_tene     para   ue no moleste lo comento

[<div class="price_container">
 <span><sup></sup>39.00</span><span>€</span> <!--<div class="incluido">(impuestos y tasas incluidos)</div>-->
 <p><span class="btn_1 tour_view_more">Ver más</span></p>
 </div>,
 <div class="price_container">
 <span><sup></sup>60.00</span><span>€</span> <!--<div class="incluido">(impuestos y tasas incluidos)</div>-->
 <p><span class="btn_1 tour_view_more">Ver más</span></p>
 </div>,
 <div class="price_container">
 <span><sup></sup>18.35</span><span>€</span> <!--<div class="incluido">(impuestos y tasas incluidos)</div>-->
 <p><span class="btn_1 tour_view_more">Ver más</span></p>
 </div>,
 <div class="price_container">
 <span><sup></sup>33.90</span><span>€</span> <!--<div class="incluido">(impuestos y tasas incluidos)</div>-->
 <p><span class="btn_1 tour_view_more">Ver más</span></p>
 </div>,
 <div class="price_container">
 <span><sup></sup>14.85</span><span>€</span> <!--<div class="incluido">(impuestos y tasas incluidos)</div>-->
 <p><span class="btn_1 tour

In [37]:
precios_tene = []

for precio in lista_precios_tene:
    precio_normal = precio.find("div", class_="normal_price")

    if precio_normal:
        precios_tene.append(precio_normal.text.strip())
    else:
        sup_precio = precio.find("sup")
        span_precio = precio.find("span")
        if span_precio:
            precios_tene.append(span_precio.text.strip())
        else:
            precios_tene.append("gratis")

In [38]:
diccionario_tene = {"actividades": actividades_tene, "categorias": categoria_activides_tene, "descripcion": descripcion_actividad_tene, "precio": precios_tene}
df_actividades_tene = pd.DataFrame(diccionario_tene)
df_actividades_tene.head()

,actividades,categorias,descripcion,precio
0,Ruta de misterios y leyendas por Santa Cruz de...,Tours a pie,"Disfruta de una fantástica noche de miedo, pai...",39.00
1,Parque Nacional del Teide y norte de la isla d...,Excursiones de un día,Nuestra primera parada será en la ciudad de La...,60.00
2,Tour por Santa Cruz de Tenerife con recogida d...,Visitas Guiadas,En esta visita podrá visitar las mejores tiend...,18.35
3,"La Laguna, Tangana y Santa Cruz desde Puerto d...",Excursiones de un día,"Descubra la belleza natural de La Laguna, Tan...",33.90
4,Visita guiada a pie por el barrio judío de San...,Tours a pie,Contemple el barrio judío de Sevilla en un to...,14.85
5,Traslados a la ciudad de Puerto de la Cruz,Excursiones de un día,Trayecto en un cómodo autobús con aire acondic...,19.25
6,"Visita guíada por Santander, 1941 el gran ince...",Tours a pie,Hay muchas maneras de conocer la capital de Ca...,10.00
7,Ruta privada de Semana Santa en Sevilla,Visitas Guiadas,Haga la fantástica ruta de Semana Santa en Sev...,160.00
8,Excursión de medio día por la ciudad de Santia...,Excursiones de un día,Salida desde su hotel en Santiago para comenza...,35.00
9,Visita guiada de la Última Cena y Santa Maria ...,Visitas Guiadas,Sigue los pasos del gran Leonardo da Vinci e...,97.00


In [24]:

driver = webdriver.Chrome()
url_wunder = "https://www.guias.travel/tour/"
driver.get(url_wunder)
driver.maximize_window()
driver.implicitly_wait(6) 

try: 
    driver.find_element("css selector", "#search-form-input").click()
except:
    print("No encuentro el botón")

driver.find_element("css selector", "#search-form-input").send_keys("mallorca", Keys.ENTER)
for i in range(5):  
    driver.execute_script("window.scrollBy(0, 1000);")  
    sleep(1) 
sleep(3)
try: 
    driver.find_element("css selector", "#more_tours > span").click()
except:
    print("No encuentro el botón")
for i in range(2):  
    driver.execute_script("window.scrollBy(0, 1000);")  
    sleep(1) 
try: 
    driver.find_element("css selector", "#more_tours > span").click()
except:
    print("No encuentro el botón")
sopa_actividades_pmi = BeautifulSoup(driver.page_source)
lista_actividades_pmi = sopa_actividades_pmi.find_all("div", {"class": "strip_all_tour_list"})
lista_actividades_pmi[0]
# div, class: 
# body > div.container.margin_60.tours_archive_container > div > div.col-lg-9.col-md-8 > div.tour-list > a:nth-child(1) > div > div


No encuentro el botón


<div class="strip_all_tour_list" data-wow-delay="0.1s">
<div class="row tour_list_item">
<div class="col-lg-4 col-md-4 col-sm-4">
<div class="img_list">
<!-- <div class="ribbon popular" ></div> -->
<img src="https://www.guias.travel/wp-content/uploads/2019/10/catedral-mallorca1-400x267.jpg" src-backup="https://www.guias.travel/wp-content/uploads/2019/10/catedral-mallorca1-400x267.jpg"/>
</div>
<h3 class="category mobile"><i class="icon-user-4"></i>Visitas Guiadas</h3>
</div>
<div class="clearfix visible-xs-block"></div>
<div class="col-lg-6 col-md-6 col-sm-6">
<div class="tour_list_desc">
<h3 class="category desktop"><i class="icon-user-4"></i>Visitas Guiadas</h3>
<h2>Visita guiada por Palma de Mallorca con entrada a la Catedral</h2>
<div class="cancelacion-gratuita">Cancelación gratuita</div>
<div class="price_list mobile">
<div class="tour_featured">
<div class="tduration">
<i class="icon-clock-3"></i>
<span>  1 hora 30 minutos</span>
</div>
<div class="tlang">
<i class="icon-comment

In [25]:
actividades_pmi = [viaje.find("h2").getText() for viaje in lista_actividades_pmi]
categoria_activides_pmi = [excursion.find("h3").getText() for excursion in lista_actividades_pmi]
descripcion_actividad_pmi = [desc.find("p").getText() for desc in lista_actividades_pmi]


lista_precios_pmi= sopa_actividades_pmi.find_all("div", {"class": "price_container"})
precio_actividades_pmi = [p.getText().strip().replace( " \nVer más", "") for p in lista_precios_pmi]

In [23]:
print(len(actividades_pmi))
print(len(categoria_activides_pmi))
print(len(descripcion_actividad_pmi))

20
20
20


In [26]:

diccionario_pmi = {"actividades": actividades_pmi, "categorias": categoria_activides_pmi, "descripcion": descripcion_actividad_pmi, "precio": precio_actividades_pmi}
df_actividades_pmi = pd.DataFrame(diccionario_pmi)

df_actividades_pmi.head()

,actividades,categorias,descripcion,precio
0,Visita guiada por Palma de Mallorca con entrad...,Visitas Guiadas,Descubra los lugares más famosos de Palma de ...,26.00€
1,Autobús turístico de Palma,Tours en Bus turístico,Visite la antigua ciudad portuaria del Mediter...,18.50€
2,"Tour por la isla: Sóller en tren de madera, tr...",Excursiones de un día,En una excursión de día completo por la parte ...,90.80€
3,Visita Palma y Alrededores – Tour en Coche Elé...,Excursiones de un día,Tendrás la oportunidad de conducir uno de nues...,65.00€
4,"Visita a las Cuevas del Drach, porto Cristo y ...",Excursiones de un día,"Excursión de medio día a Majorica, Porto Crist...",50.60€
5,Catamarán Magic a Es Trenc desde Colonia de Sa...,Paseos en barco,Zarpe hacia un día mágico en el mar mientras s...,56.00€
6,Tour por Palma de Mallorca y Valldemossa,Excursiones de un día,Visite la Ciudad de Palma de Mallorca y su Cat...,53.00€
7,"Tour de un día a las Cuevas del Drach, Majoric...",Excursiones de un día,Descubra los encantos de la costa este de Mall...,69.00€
8,Menorcalines por Trasmapi: Ferri de Mallorca a...,Paseos en barco,"Reserve este billete de ferry, hacia la maravi...",82.00€
9,Tour por la Cueva dels Hams y la Fabrica de pe...,Tours a pie,Disfrute de una visita excepcional a la Cueva ...,47.00€
